In [93]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
import os

from sklearn.ensemble import RandomForestRegressor

#Evaluation:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
import ast
from geopy import distance
from sklearn.model_selection import RandomizedSearchCV

import utils
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [94]:
def process_fixes(df):
    df['decoded_fixes'] = df['decoded_fixes'].apply(lambda x: ast.literal_eval(x))
    
    df.loc[:, 'lat_start'] = df['decoded_fixes'].map(lambda x: x[0][0])
    df.loc[:, 'lng_start'] = df['decoded_fixes'].map(lambda x: x[0][1])
    
    df.loc[:, 'lat_end'] = df['decoded_fixes'].map(lambda x: x[-1][0])
    df.loc[:, 'lng_end'] = df['decoded_fixes'].map(lambda x: x[-1][1])
    
    df.drop(columns=['decoded_fixes'], inplace=True)

In [95]:
df_train = pd.read_csv('data/flight_plans_train.csv')
df_train.drop(columns=['departure_time','id', 'departure_airport', 'arrival_airport', 'fixes'], inplace=True)
df_train['Distance'] = df_train.apply(lambda x: utils.dist_in_miles_from_spherical_path(ast.literal_eval(x['decoded_fixes'])), axis=1)
process_fixes(df_train)
df_train['DistanceStraight'] = df_train.apply(lambda x: distance.distance((x['lat_start'], x['lng_start']), (x['lat_end'], x['lng_end'])).m, axis=1)
df_train['DsitancesProduct'] = df_train['Distance'] * df_train['DistanceStraight']
df_train['TimeFromDistanceStraight'] = df_train['DistanceStraight'] / df_train['requested_airspeed']
df_train['TimeFromDistance'] = df_train['Distance'] / df_train['requested_airspeed']

In [96]:
df_train.head()

,assigned_altitude,requested_airspeed,Distance,lat_start,lng_start,lat_end,lng_end,DistanceStraight,DsitancesProduct,TimeFromDistanceStraight,TimeFromDistance
0,29000,469.0,351.472012,44.881972,-93.221778,41.974522,-87.906597,5.379240e+05,1.890652e+08,1146.959569,0.749407
1,34000,461.0,1762.130590,41.974522,-87.906597,33.942494,-118.408050,2.807409e+06,4.947021e+09,6089.824493,3.822409
2,31000,438.0,571.232884,46.920639,-96.815750,41.974522,-87.906597,8.962677e+05,5.119776e+08,2046.273249,1.304185
3,32000,452.0,541.254583,41.974522,-87.906597,36.281581,-94.307767,8.396074e+05,4.544414e+08,1857.538561,1.197466
4,32000,467.0,465.057792,41.974522,-87.906597,39.297611,-94.713889,6.477913e+05,3.012604e+08,1387.133330,0.995841


In [97]:
    wheathers = pd.read_csv('weather_features.csv')
    df_train = pd.concat([df_train, wheathers], axis=1, sort=False)
    df_train = df_train.replace(-999, 0)
    #df_train = df_train.fillna(0.0)

In [98]:
df_train.head()

,assigned_altitude,requested_airspeed,Distance,lat_start,lng_start,lat_end,lng_end,DistanceStraight,DsitancesProduct,TimeFromDistanceStraight,TimeFromDistance,cloud_echo_tops_0_0,vertically_integrated_liquid_water_0_0,flight_level_25000_u_0_0,flight_level_25000_v_0_0,flight_level_30000_u_0_0,flight_level_30000_v_0_0,flight_level_50000_u_0_0,flight_level_50000_v_0_0,cloud_echo_tops_1_0,vertically_integrated_liquid_water_1_0,flight_level_25000_u_1_0,flight_level_25000_v_1_0,flight_level_30000_u_1_0,flight_level_30000_v_1_0,flight_level_50000_u_1_0,flight_level_50000_v_1_0,cloud_echo_tops_2_0,vertically_integrated_liquid_water_2_0,flight_level_25000_u_2_0,flight_level_25000_v_2_0,flight_level_30000_u_2_0,flight_level_30000_v_2_0,flight_level_50000_u_2_0,flight_level_50000_v_2_0,cloud_echo_tops_3_0,vertically_integrated_liquid_water_3_0,flight_level_25000_u_3_0,flight_level_25000_v_3_0,flight_level_30000_u_3_0,flight_level_30000_v_3_0,flight_level_50000_u_3_0,flight_level_50000_v_3_0,cloud_echo_tops_4_0,vertically_integrated_liquid_water_4_0,flight_level_25000_u_4_0,flight_level_25000_v_4_0,flight_level_30000_u_4_0,flight_level_30000_v_4_0,flight_level_50000_u_4_0,flight_level_50000_v_4_0,cloud_echo_tops_0_1,vertically_integrated_liquid_water_0_1,flight_level_25000_u_0_1,flight_level_25000_v_0_1,flight_level_30000_u_0_1,flight_level_30000_v_0_1,flight_level_50000_u_0_1,flight_level_50000_v_0_1,cloud_echo_tops_1_1,vertically_integrated_liquid_water_1_1,flight_level_25000_u_1_1,flight_level_25000_v_1_1,flight_level_30000_u_1_1,flight_level_30000_v_1_1,flight_level_50000_u_1_1,flight_level_50000_v_1_1,cloud_echo_tops_2_1,vertically_integrated_liquid_water_2_1,flight_level_25000_u_2_1,flight_level_25000_v_2_1,flight_level_30000_u_2_1,flight_level_30000_v_2_1,flight_level_50000_u_2_1,flight_level_50000_v_2_1,cloud_echo_tops_3_1,vertically_integrated_liquid_water_3_1,flight_level_25000_u_3_1,flight_level_25000_v_3_1,flight_level_30000_u_3_1,flight_level_30000_v_3_1,flight_level_50000_u_3_1,flight_level_50000_v_3_1,cloud_echo_tops_4_1,vertically_integrated_liquid_water_4_1,flight_level_25000_u_4_1,flight_level_25000_v_4_1,flight_level_30000_u_4_1,flight_level_30000_v_4_1,flight_level_50000_u_4_1,flight_level_50000_v_4_1,cloud_echo_tops_0_2,vertically_integrated_liquid_water_0_2,flight_level_25000_u_0_2,flight_level_25000_v_0_2,flight_level_30000_u_0_2,flight_level_30000_v_0_2,flight_level_50000_u_0_2,flight_level_50000_v_0_2,cloud_echo_tops_1_2,vertically_integrated_liquid_water_1_2,flight_level_25000_u_1_2,flight_level_25000_v_1_2,flight_level_30000_u_1_2,flight_level_30000_v_1_2,flight_level_50000_u_1_2,flight_level_50000_v_1_2,cloud_echo_tops_2_2,vertically_integrated_liquid_water_2_2,flight_level_25000_u_2_2,flight_level_25000_v_2_2,flight_level_30000_u_2_2,flight_level_30000_v_2_2,flight_level_50000_u_2_2,flight_level_50000_v_2_2,cloud_echo_tops_3_2,vertically_integrated_liquid_water_3_2,flight_level_25000_u_3_2,flight_level_25000_v_3_2,flight_level_30000_u_3_2,flight_level_30000_v_3_2,flight_level_50000_u_3_2,flight_level_50000_v_3_2,cloud_echo_tops_4_2,vertically_integrated_liquid_water_4_2,flight_level_25000_u_4_2,flight_level_25000_v_4_2,flight_level_30000_u_4_2,flight_level_30000_v_4_2,flight_level_50000_u_4_2,flight_level_50000_v_4_2,cloud_echo_tops_0_3,vertically_integrated_liquid_water_0_3,flight_level_25000_u_0_3,flight_level_25000_v_0_3,flight_level_30000_u_0_3,flight_level_30000_v_0_3,flight_level_50000_u_0_3,flight_level_50000_v_0_3,cloud_echo_tops_1_3,vertically_integrated_liquid_water_1_3,flight_level_25000_u_1_3,flight_level_25000_v_1_3,flight_level_30000_u_1_3,flight_level_30000_v_1_3,flight_level_50000_u_1_3,flight_level_50000_v_1_3,cloud_echo_tops_2_3,vertically_integrated_liquid_water_2_3,flight_level_25000_u_2_3,flight_level_25000_v_2_3,flight_level_30000_u_2_3,flight_level_30000_v_2_3,flight_level_50000_u_2_3,flight_level_50000_v_2_3,cloud_echo_tops_3_3,vertically_integrated_liquid_water_3_3,flight_level_25000_u

In [99]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(df_train)

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='mean', verbose=0)

In [100]:
    Y = pd.read_csv('data/flight_times_labels_train.csv')['flight_time_s'].astype(float).values.reshape(-1)
    X = df_train.values
    X = imp.transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=5)
    
    min_max_scaler = preprocessing.MinMaxScaler()
    X_train = min_max_scaler.fit_transform(X_train)
    X_test = min_max_scaler.transform(X_test)

In [83]:
model = RandomForestRegressor(n_estimators=200)

In [84]:
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=200, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [85]:
    pred = model.predict(X_test).flatten()
    
    outcomes = pd.DataFrame({'Prediction': pred.tolist(), 'Ground_Truth': y_test.tolist(), 'Difference': (y_test - pred).tolist()})
    print(outcomes.describe())

Prediction  Ground_Truth   Difference
count   5368.000000   5368.000000  5368.000000
mean    6146.075882   6136.449329    -9.626553
std     3194.999719   3213.165030   297.205703
min     1579.280000   1505.000000 -2198.205000
25%     3705.035000   3708.000000  -157.830000
50%     5527.472500   5484.000000   -30.647500
75%     7196.240000   7270.250000   114.178750
max    15714.835000  16610.000000  6119.750000


In [103]:
model = pickle.load(open('finalized_model500.sav', 'rb'))

In [101]:
    test_data = pd.read_csv('data/flight_plans_test.csv')
    test_ids = test_data[['id']].squeeze()
    test_data.drop(columns=['id', 'departure_airport', 'arrival_airport', 'fixes', 'departure_time'], inplace=True)
    test_data['Distance'] = test_data.apply(lambda x: utils.dist_in_miles_from_spherical_path(ast.literal_eval(x['decoded_fixes'])), axis=1)
    process_fixes(test_data)
    test_data['DistanceStraight'] = test_data.apply(lambda x: distance.distance((x['lat_start'], x['lng_start']), (x['lat_end'], x['lng_end'])).m, axis=1)
    test_data['DsitancesProduct'] = test_data['Distance'] * test_data['DistanceStraight']
    test_data['TimeFromDistanceStraight'] = test_data['DistanceStraight'] / test_data['requested_airspeed']
    test_data['TimeFromDistance'] = test_data['Distance'] / test_data['requested_airspeed']
    
    wheathers_test = pd.read_csv('weather_features_test.csv')
    test_data = pd.concat([test_data, wheathers_test], axis=1, sort=False)
    test_data = test_data.replace(-999, 0)


In [104]:
X_test_data = test_data.values
X_test_data = imp.transform(X_test_data)
X_test_data = min_max_scaler.transform(X_test_data)
pred = model.predict(X_test_data).flatten()
    
test_outcome = pd.DataFrame({'id': list(test_ids), 'flight_time_s': list(pred.astype(int))})

In [105]:
test_outcome.to_csv('results4.csv', index=False)

In [14]:
import pickle
# save the classifier
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))    

In [15]:
loaded_model = pickle.load(open(filename, 'rb'))

NameError: name 'Y_test' is not defined

In [16]:
loaded_model

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=30, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)